In [ ]:
from sklearn.cluster import DBSCAN, KMeans
import numpy as np
from collections import defaultdict
from network import Network
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d


def create_cluster_nodes(network: Network, n_clusters: int = 2):
    """
    Clusters nodes in the given network using DBSCAN.
    
    Args:
        network (Network): The network to be clustered.
        eps (float): The maximum distance between two samples for them to be considered in the same neighborhood.
        min_samples (int): The number of samples (or total weight) in a neighborhood for a point to be considered as a core point.

    Returns:
        dict[int, list[Node]]: A dictionary mapping cluster IDs to lists of nodes in that cluster.
    """
    # Extract positions as a 2D arraynetwork = Network.from_dicts(pings_csv_to_dict("./../pings.csv"), servers_csv_to_dict("./../servers.csv"), fraction = 1)
    positions = np.array([[node.pos.x, node.pos.y] for node in network.nodes])

    # Run DBSCAN clustering
    kmeans = KMeans(n_clusters = n_clusters)
    clustering = kmeans.fit(positions)

    # Create a dictionary mapping clusters to nodes
    cluster_map = defaultdict(list)
    for node, cluster_label in zip(network.nodes, clustering.labels_):
        cluster_map[cluster_label].append(node)

    # Remove the noise cluster (label == -1) if present
    if -1 in cluster_map:
        del cluster_map[-1]
        
    

    # Calculate clusters centroids
    centroids = list()
    for cluster_nodes in cluster_map.values():
        positions = np.array([[x, y] for x, y in [(node.pos.x, node.pos.y) for node in cluster_nodes]])
        centroids.append(np.mean(positions, axis=0))

    # Voronoi tessellation
    centroids = np.array(centroids)[:, [1, 0]]
    # Extend centroids for wrapping at x = ±180
    extended_centroids = np.vstack([
        centroids,  # Original
        centroids + np.array([360, 0]),  # Shifted right (wrap-around)
        centroids - np.array([360, 0])   # Shifted left (wrap-around)
    ])
    vor = Voronoi(extended_centroids)


    x_min, x_max = -180, 180
    y_min, y_max = -90, 90

    neighbors = {int(cluster_id): set() for cluster_id in range(len(centroids))}
    
    # Iterate through ridge_points to find neighbors
    for ridge, point_pair in zip(vor.ridge_vertices, vor.ridge_points):
    # Get the vertices of the ridge (edge)
        inside = False
        vertex_coords = [vor.vertices[i] for i in ridge if i != -1]
        for v in vertex_coords:
            if (x_min <= v[0] <= x_max and y_min <= v[1] <= y_max):
                inside = True
                break
        if not inside:
            continue

        cluster_a = point_pair[0] % len(centroids)
        cluster_b = point_pair[1] % len(centroids)
        neighbors[cluster_a].add(int(cluster_b))
        neighbors[cluster_b].add(int(cluster_a))
    
    for cluster_id, neighbor_set in neighbors.items():
        print(f"Cluster {cluster_id} neighbors: {neighbor_set}")
    
    fig = plt.figure(figsize=(9 / 2.54, 6 / 2.54))
    ax = fig.add_subplot(111)
    voronoi_plot_2d(vor, ax=ax, show_points=False, show_vertices=False, line_colors='orange')
    
    y = [node.pos.x for node in network.nodes]
    x = [node.pos.y for node in network.nodes]

    plt.scatter(x, y, c = kmeans.labels_, s=1)
    plt.scatter([centroid[0] for centroid in centroids], [centroid[1] for centroid in centroids], c = 'red', s=3)
    for i, (cx, cy) in enumerate(centroids):
            plt.text(cx, cy, f"{i}", color="black", fontsize=12, ha="center", va="center", bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.2'))
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.grid()
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    plt.tight_layout()  # Adjust layout before saving
    plt.savefig("voronoi.png", dpi=300)
    plt.show()


    return cluster_map, kmeans.inertia_, centroids

In [ ]:
from simulator import Simulator
from network import Network
import numpy as np

from network import servers_csv_to_dict, pings_csv_to_dict

network = Network.from_dicts(pings_csv_to_dict("./../pings.csv"), servers_csv_to_dict("./../servers.csv"), fraction = 1)


In [ ]:
# Cluster nodes
clusters, inertia_v, centroids = create_cluster_nodes(network, n_clusters=8)
network.show_network()

# Print cluster assignments
for cluster_id, cluster_nodes in clusters.items():
    print(f"Cluster {cluster_id}: {[node.node_id for node in cluster_nodes]}")